In [1]:
import QuantLib as ql
import pandas as pd
from datetime import date
from utils import *

In [2]:
banking_params = connParams.copy()
banking_params['database'] = 'Bolsa'
banking_params['schema'] = 'basesbalance'

curves_params = connParams

banking_conn = connectDB(**banking_params)
curves_conn = connectDB(**curves_params)

### Data from db

In [3]:
def term_translate(term):
    if 'O/N' in term or 'DY' in term:
        return str(ql.Period(1, ql.Days))
    elif 'MO' in term:
        tmp = term.replace('MO', '')
        return str(ql.Period(int(tmp), ql.Months))
    elif 'WK' in term:
        tmp = term.replace('WK', '')
        return str(ql.Period(int(tmp), ql.Weeks))
    elif 'YR' in term:
        tmp = term.replace('YR', '')
        return str(ql.Period(int(tmp), ql.Years))

def helper_type_translate(helper_type):
    return helper_type.upper()


In [8]:
sql = 'select * from rate_helper_params'
df = pd.read_sql(sql, banking_conn)
df = df[df.curve_name=='ICP']

tickers = df.bbg_ticker.to_list()
terms = df.term.to_list()
helper_types = df.rate_helper.to_list()

terms = list(map(term_translate, terms))
helper_types = list(map(helper_type_translate, helper_types))

In [9]:
add_tickers(tickers, curves_conn)

True

In [35]:
curves_params = {
    'curve_name': 'CF_USD',
    'day_counter': 'Act360',
    'enable_extrapolation': True,
    'config_date': date(2022, 7, 26),
}

helper_param = {
    'helper_name': 'EUR-EURIBOR-6M',
    'type':'DEPOSIT',
    'fields':['rate','tenor'],
    'values':[]
}
deposit_fields = ['rate', 'tenor', 'dayCounter']
bond_fields = ['rate', 'tenor', 'dayCounter', 'fixingDays', 'index',' discountingCurve']
swap_fields = ['rate', 'tenor', 'dayCounter']
ois_fields = ['tenor', 'rate', 'index', 'discountingCurve', 'frequency']


### upload ICP

In [45]:

curves_params = {
    'curve_name': 'ICP_ICAP',
    'day_counter': 'Act360',
    'enable_extrapolation': True,
    'config_date': str(date(2022, 7, 26)),
}

curve_name = curves_params['curve_name']
helpers = []

process = [] 
for term, helper_type, ticker in zip(terms, helper_types, tickers):     
    if 'CMPN' not in ticker and term not in process:
        process.append(term)
        helper = {
            'helper_name': curve_name + '_' + term,
            'type': helper_type,
            'fields': None,
            'values': None
        }   
        if helper_type == 'DEPOSIT':
            helper['type'] = 'OIS'
            helper['fields'] = ois_fields
            helper['values'] = [term, ticker, 'ICPIndex', 'CLP_COLLUSD', 'ONCE']
        elif helper_type == 'OIS':
            helper['fields'] = ois_fields
            helper['values'] = [term, ticker, 'ICPIndex', 'CLP_COLLUSD', 'SEMIANNUAL']    
        elif helper_type == 'SWAP':
            helper['type'] = 'OIS'
            helper['fields'] = ois_fields
            helper['values'] = [term, ticker, 'ICPIndex', 'CLP_COLLUSD', 'SEMIANNUAL']  
        helpers.append(helper)      

upload_curve(curves_params, helpers, curves_conn) 

{'curve_name': 'ICP_ICAP', 'day_counter': 'Act360', 'enable_extrapolation': True, 'config_date': '2022-07-26'}


True

#### upload SOFR

In [16]:
sql = 'select * from rate_helper_params'
df = pd.read_sql(sql, banking_conn)
df = df[df.curve_name=='SOFR']

tickers = df.bbg_ticker.to_list()
terms = df.term.to_list()
helper_types = df.rate_helper.to_list()

terms = list(map(term_translate, terms))
helper_types = list(map(helper_type_translate, helper_types))

In [14]:
delete_curve(get_curve_id(curves_conn, 'SOFR'), curves_conn)

True

In [7]:
add_tickers(tickers, curves_conn)

True

In [17]:
deposit_fields = ['rate', 'tenor', 'dayCounter']
ois_fields = ['tenor', 'rate', 'index', 'frequency', 'calendar']

curves_params = {
    'curve_name': 'SOFR',
    'day_counter': 'Act360',
    'enable_extrapolation': True,
    'config_date': str(date(2022, 7, 25)),
}

curve_name = curves_params['curve_name']
helpers = []

process = [] 
for term, helper_type, ticker in zip(terms, helper_types, tickers):     
    if 'CMPN' not in ticker and term not in process:
        process.append(term)
        helper = {
            'helper_name': curve_name + '_' + term,
            'type': helper_type,
            'fields': None,
            'values': None
        }   
        if helper_type == 'DEPOSIT':
            helper['fields'] = deposit_fields
            helper['values'] = [ticker, term, 'Act360']
        elif helper_type == 'OIS':
            helper['fields'] = ois_fields
            helper['values'] = [term, ticker, 'SOFR', 'ANNUAL', 'USA']    
        helpers.append(helper)      

upload_curve(curves_params, helpers, curves_conn) 

{'curve_name': 'SOFR', 'day_counter': 'Act360', 'enable_extrapolation': True, 'config_date': '2022-07-25'}


True

### upload LIBOR3M

In [18]:
sql = 'select * from rate_helper_params'
df = pd.read_sql(sql, banking_conn)
df = df[df.curve_name=='LIBOR3M']

tickers = df.bbg_ticker.to_list()
terms = df.term.to_list()
helper_types = df.rate_helper.to_list()

terms = list(map(term_translate, terms))
helper_types = list(map(helper_type_translate, helper_types))

In [20]:
deposit_fields = ['rate', 'tenor', 'dayCounter']
swap_fields = ['rate','tenor','index', 'frequency', 'discountingCurve', 'calendar']

curves_params = {
    'curve_name': 'LIBOR3M',
    'day_counter': 'Act360',
    'enable_extrapolation': True,
    'config_date': str(date(2022, 7, 25)),
}

curve_name = curves_params['curve_name']
helpers = []

process = [] 
ticks = []
for term, helper_type, ticker in zip(terms, helper_types, tickers):     
    if 'Comdty' not in ticker and term not in process:
        process.append(term)
        ticks.append(ticker)
        helper = {
            'helper_name': curve_name + '_' + term,
            'type': helper_type,
            'fields': None,
            'values': None
        }   
        if helper_type == 'DEPOSIT':
            helper['fields'] = deposit_fields
            helper['values'] = [ticker, term, 'Act360']
        elif helper_type == 'SWAP':
            helper['fields'] = swap_fields
            helper['values'] = [ticker, term, 'LIBOR3M', 'SEMIANNUAL', 'SOFR', 'USA']    
        helpers.append(helper)      

#

In [21]:
upload_curve(curves_params, helpers, curves_conn) 

{'curve_name': 'LIBOR3M', 'day_counter': 'Act360', 'enable_extrapolation': True, 'config_date': '2022-07-25'}


True

### upload XCCY

In [7]:
sql = 'select * from rate_helper_params'
df = pd.read_sql(sql, banking_conn)
df = df[df.curve_name=='COLLATERAL_USDCLP']

tickers = df.bbg_ticker.to_list()
terms = df.term.to_list()
helper_types = df.rate_helper.to_list()

terms = list(map(term_translate, terms))
helper_types = list(map(helper_type_translate, helper_types))

In [8]:
fxswap_fields = ['FXPOINTS','SPOTFX', 'TENOR', 'CALENDAR', 'COLLATERALCURVE','ENDOFMONTH','BASECURRENCYCOLLATERAL']
xccy_fields = ['RATE','SPOTFX','TENOR','INDEX', 'FREQUENCY', 'DISCOUNTINGCURVE', 'CALENDAR']

curves_params = {
    'curve_name': 'CLP_COLLUSD',
    'day_counter': 'ACT360',
    'enable_extrapolation': True,
    'config_date': str(date(2022, 7, 25)),
}

curve_name = curves_params['curve_name']
helpers = []

process = [] 
ticks = []
for term, helper_type, ticker in zip(terms, helper_types, tickers):     
    if 'Comdty' not in ticker and term not in process:
        process.append(term)
        ticks.append(ticker)
        helper = {
            'helper_name': curve_name + '_' + term,
            'type': helper_type,
            'fields': None,
            'values': None
        }   
        if helper_type == 'FXSWAP':
            helper['fields'] = fxswap_fields
            helper['values'] = [ticker, term, 'Act360']
        elif helper_type == 'XCCY':
            helper['fields'] = xccy_fields
            helper['values'] = [ticker, term, 'LIBOR3M', 'SEMIANNUAL', 'SOFR', 'USA']    
        helpers.append(helper)   

,idx,curve_name,bbg_ticker,day_counter,term,instrument_type,frequency,rate_helper,buss_adj,eom,payment_lag,settlement_days,calendar,fix_base_ticker,base_as_collateral
130,1.0,COLLATERAL_USDCLP,CHN1W Curncy,ACT_360_EOMC,1WK,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
131,2.0,COLLATERAL_USDCLP,CHN2W Curncy,ACT_360_EOMC,2 WK,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
132,3.0,COLLATERAL_USDCLP,CHN1M Curncy,ACT_360_EOMC,1 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
133,4.0,COLLATERAL_USDCLP,CHN2M Curncy,ACT_360_EOMC,2 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
134,5.0,COLLATERAL_USDCLP,CHN3M Curncy,ACT_360_EOMC,3 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
135,6.0,COLLATERAL_USDCLP,CHN4M Curncy,ACT_360_EOMC,4 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
136,7.0,COLLATERAL_USDCLP,CHN5M Curncy,ACT_360_EOMC,5 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
137,8.0,COLLATERAL_USDCLP,CHN6M Curncy,ACT_360_EOMC,6 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
138,9.0,COLLATERAL_USDCLP,CHN9M Curncy,ACT_360_EOMC,9 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
139,10.0,COLLATERAL_USDCLP,CHN12M Curncy,ACT_360_EOMC,12 MO,FXSWAP,ZERO,FxSwap,ModFollowing,True,0.0,0.0,UnitedStates,None,True
